In [31]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from rich import print
from rich.pretty import pprint

FILENAME = "train.csv"
df = pd.read_csv(FILENAME)



X = df.drop("Year", axis=1)
y = df["Year"]

nomi_colonne = list(X.columns)
print(nomi_colonne)

# Esegui lo split in set di addestramento, di test e di validazione
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, random_state=42, test_size=0.2)
train, val = train_test_split(train, random_state=42, test_size=0.2)
print(f"Train Shape: {train.shape} | Val Shape: {val.shape} | Test Shape: {test.shape}")

[
    'S0',
    'S1',
    'S2',
    'S3',
    'S4',
    'S5',
    'S6',
    'S7',
    'S8',
    'S9',
    'S10',
    'S11',
    'S12',
    'S13',
    'S14',
    'S15',
    'S16',
    'S17',
    'S18',
    'S19',
    'S20',
    'S21',
    'S22',
    'S23',
    'S24',
    'S25',
    'S26',
    'S27',
    'S28',
    'S29',
    'S30',
    'S31',
    'S32',
    'S33',
    'S34',
    'S35',
    'S36',
    'S37',
    'S38',
    'S39',
    'S40',
    'S41',
    'S42',
    'S43',
    'S44',
    'S45',
    'S46',
    'S47',
    'S48',
    'S49',
    'S50',
    'S51',
    'S52',
    'S53',
    'S54',
    'S55',
    'S56',
    'S57',
    'S58',
    'S59',
    'S60',
    'S61',
    'S62',
    'S63',
    'S64',
    'S65',
    'S66',
    'S67',
    'S68',
    'S69',
    'S70',
    'S71',
    'S72',
    'S73',
    'S74',
    'S75',
    'S76',
    'S77',
    'S78',
    'S79',
    'S80',
    'S81',
    'S82',
    'S83',
    'S84',
    'S85',
    'S86',
    'S87',
    'S88',
    'S89'
]

Train Shape: (161392, 91) | Val Shape: (40348, 91) | Test Shape: (50435, 91)

In [32]:
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
)
from pytorch_tabular.models.common.heads import LinearHeadConfig

data_config = DataConfig(
    target=["Year"],
    continuous_cols= nomi_colonne,
    categorical_cols=[],
)
trainer_config = TrainerConfig(
    batch_size=1024,
    max_epochs=25,
    auto_lr_find=True,
    early_stopping=None,  # Monitor valid_loss for early stopping
    # early_stopping_mode="min",  # Set the mode as min because for val_loss, lower is better
    # early_stopping_patience=5,  # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss",  # Save best checkpoint monitoring val_loss
    load_best=True,  # After training, load the best checkpoint
    progress_bar="none",  # Turning off Progress bar
    trainer_kwargs=dict(enable_model_summary=False),  # Turning off model summary
    accelerator="cpu",
)
optimizer_config = OptimizerConfig()

head_config = LinearHeadConfig(
    layers="",
    dropout=0.1,
    initialization=(  # No additional layer in head, just a mapping layer to output_dim
        "kaiming"
    ),
).__dict__  # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

In [33]:
from pytorch_tabular import MODEL_SWEEP_PRESETS

print(list(MODEL_SWEEP_PRESETS.keys()))

['lite', 'standard', 'full', 'high_memory']

In [34]:
pprint(list(MODEL_SWEEP_PRESETS["full"]))

[]

In [38]:
from pytorch_tabular.models import TabNetModelConfig, TabTransformerConfig
from pytorch_tabular import model_sweep
import warnings
common_params = {
    "task": "regression",
    "head":"LinearHead", "head_config":head_config
}
model_list = [
    TabNetModelConfig(n_d= 32, n_a= 32, n_steps=3, gamma=1.5, n_independent= 1, n_shared= 2, **common_params),
    TabTransformerConfig(**common_params),
]

# Filtering out the warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sweep_df, best_model = model_sweep(
        task="regression",  # One of "classification", "regression"
        train=train,
        test=test,
        data_config=data_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        model_list=model_list,
        progress_bar=True,
        verbose=False,
        suppress_lightning_logger=True,
    )

Output()

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

In [39]:
sweep_df

,model,# Params,epochs,test_loss,test_mean_squared_error,time_taken,time_taken_per_epoch,params
1,TabTransformerModel,271 T,25,98.116257,98.116257,45.991377,1.839655,"{'task': 'regression', 'head': 'LinearHead', '..."
0,TabNetModel,11 T,25,127.765312,127.765312,232.025002,9.281000,"{'task': 'regression', 'head': 'LinearHead', '..."


In [37]:
sweep_df.drop(columns=["params", "time_taken", "epochs"]).style.background_gradient(
    subset=["test_mean_squared_error"], cmap="RdYlGn"
).background_gradient(subset=["time_taken_per_epoch", "test_loss"], cmap="RdYlGn_r")

,model,# Params,test_loss,test_mean_squared_error,time_taken_per_epoch
2,TabTransformerModel,271 T,97.847755,97.847755,1.903631
0,TabNetModel,65 T,115.400337,115.400337,10.850818
1,TabNetModel,11 T,119.003838,119.003838,9.465802
